In [59]:
import boto3
import json
import mysql.connector
from datetime import datetime as dt

# Create JSON from IVR DynamoDB Table
This code will take in data from the contactDetails DynamoDB table and format it into a JSON file so that it can be read and processed into a dashboard for analysis purposes

In [60]:
profile = "default"
boto3 = boto3.session.Session(profile_name="fcv")

## Define DynamoDB Table Parameters
Setup for the contactDetails dynamodb table and scan to grab all the items

In [61]:
dynamodb = boto3.resource("dynamodb")
table_name = "contactDetails"
contact_details = dynamodb.Table(table_name)
calls = contact_details.scan()

## Sort by Date
This function will return the date of the feedback as a datetime object in order to sort using python's sort() function

In [62]:
def sort_by_date(feedback):
    return dt.strptime(feedback["callDate"], "%m/%d/%Y")

## Get Calls
This function will loop through the items in the contactDetails table and will count the calls that fall under a specific category. Any calls that cannot be categorized under a specific category are counted as unknown

In [63]:
def get_calls(calls):
    types_of_feedback = {
        "serviceFeedback": 0,
        "doctorFeedback": 0,
        "medFeedback": 0,
        "otherFeedback": 0, 
        "unknownFeedback": 0,
    }
    
    for call in calls["Items"]:
        audio_to_customer = call["contactTranscriptToCustomer"].lower()
        if "you have pressed one" in audio_to_customer or "you have press one" in audio_to_customer:
            types_of_feedback["serviceFeedback"] += 1
            call["feedbackType"] = "serviceFeedback"
        elif "you have pressed two" in audio_to_customer or "you have press two" in audio_to_customer:
            types_of_feedback["doctorFeedback"] += 1
            call["feedbackType"] = "doctorFeedback"
        elif "you have pressed three" in audio_to_customer or "you have press three" in audio_to_customer:
            types_of_feedback["medFeedback"] += 1
            call["feedbackType"] = "medFeedback"
        elif "you have pressed four" in audio_to_customer or "you have press four" in audio_to_customer:
            types_of_feedback["otherFeedback"] += 1
            call["feedbackType"] = "otherFeedback"
        else:
            types_of_feedback["unknownFeedback"] += 1
            call["feedbackType"] = "unknownFeedback"
        
        # Encode the urls so that they can be publicly accessible
        call["audioFromCustomer"] = encode_s3_url(call["audioFromCustomer"])
        call["audioToCustomer"] = encode_s3_url(call["audioToCustomer"])
    
    return (calls, types_of_feedback)

## Chatbot Messages RDS Query
This function will take in the call data using mysql.connector to pull from an RDS Instance

In [64]:
def get_messages(types_of_feedback):
    try:
        mydb = mysql.connector.connect(
            host="chatbot-response.cxpevijcux2x.us-east-1.rds.amazonaws.com",
            user="admin",
            password="FxUIMZYmCRYJvnJtj7fL",
            database='chatbot'
        )
        
        # Cursor object to query data from database
        mycursor = mydb.cursor()

        # Define a query
        query = "SELECT Phone, FeedbackType, Feedback, max(Created) FROM chatbot.Messages Where feedback is not NULL group by Phone, feedbacktype, Feedback"

        # Execute the query using the cursor
        mycursor.execute(query)

        # Collect the results of the query in a variable
        myresult = mycursor.fetchall()

        messagers = []
        for feedback in myresult:
            messager = {}
            
            # Get the phone number of the messager
            messager["customerPhoneNumber"] = feedback[0]
            
            # Categorize messages into their feedback categories
            if feedback[1] == 1:
                types_of_feedback["serviceFeedback"] += 1
                messager["feedbackType"] = "serviceFeedback"
            elif feedback[1] == 2:
                types_of_feedback["doctorFeedback"] += 1
                messager["feedbackType"] = "doctorFeedback"
            elif feedback[1] == 3:
                types_of_feedback["medFeedback"] += 1
                messager["feedbackType"] = "medFeedback"
            elif feedback[1] == 4:
                types_of_feedback["otherFeedback"] += 1
                messager["feedbackType"] = "otherFeedback"
            else:
                types_of_feedback["unknownFeedback"] += 1
                messager["feedbackType"] = "unknownFeedback"
            
            # Get the transcript from customer
            messager["contactTranscriptFromCustomer"] = feedback[2]


            # Get the timestamp
            messager["messageTimestamp"] = feedback[3].strftime("%c")

            # Get the message date
            messager["callDate"] = f"{feedback[3].month}/{feedback[3].day}/{feedback[3].year}"

            # Add messager to messagers array
            messagers.append(messager)

        # Close the database connection
        mydb.close()

        # Sort messages by date in descending order
        messagers.sort(key=sort_by_date, reverse=True)

        return (messagers, types_of_feedback)
    except mysql.connector.Error as err:
        print("Something went wrong: {}".format(err))



## Get Dates
This function will loop through the items in the contactDetails table and will grab the individual dates the callers contacted the IVR system

In [65]:
def get_dates(feedback):
    dates = []

    for response in feedback:
        if response['callDate'] not in dates:
            dates.append(response['callDate'])

    return dates


## Get Feedback Per Day
This function will loop through a given list of dates and will match the dates with the calls and messages. This will output a list of total feedback received during a specific date

In [66]:
def get_feedback_per_day(feedback, dates):
    feedback_per_day = []

    for date in dates:
        num = 0
        for response in feedback:
            if date == response['callDate']:
                num += 1
        feedback_per_day.append(num)

    return feedback_per_day

## Get Types of Feedback Per Day
This function will loop through a given list of dates and will match the dates with the types of feedback in the calls and messages. This will output a dict with lists representing each type of feedback and the number of responses made during a specific date

In [67]:
def get_types_of_feedback_per_day(feedback, dates):
    types_of_feedback = {
        "serviceFeedback": [],
        "doctorFeedback": [],
        "medFeedback": [],
        "otherFeedback": [], 
        "unknownFeedback": [],
    }

    for date in dates:
        num = {
            "serviceFeedback": 0,
            "doctorFeedback": 0,
            "medFeedback": 0,
            "otherFeedback": 0,
            "unknownFeedback": 0
        }
        for response in feedback:
            if response['feedbackType'] == "serviceFeedback" and response['callDate'] == date:
                num["serviceFeedback"] += 1
            elif response['feedbackType'] == "doctorFeedback" and response['callDate'] == date:
                num["doctorFeedback"] += 1
            elif response['feedbackType'] == "medFeedback" and response['callDate'] == date:
                num["medFeedback"] += 1
            elif response['feedbackType'] == "otherFeedback" and response['callDate'] == date:
                num["otherFeedback"] += 1
            elif response['feedbackType'] == "unknownFeedback" and response['callDate'] == date:
                num["unknownFeedback"] += 1
        
        for feedback_type in list(types_of_feedback.keys()):
            types_of_feedback[feedback_type].append(num[feedback_type])

    return types_of_feedback
    


## S3 URL Encode
This function will take in an s3 url and encode it so that it is publicly accessible, credits to: [George Phillips](https://github.com/GeorgePhillips/node-s3-url-encode/blob/master/index.js)

In [68]:
encodings = {
  '+': "%2B",
  '!': "%21",
  '\"': "%22",
  '#': "%23",
  '$': "%24",
  '&': "%26",
  '\'': "%27",
  '(': "%28",
  ')': "%29",
  '*': "%2A",
  ',': "%2C",
  ':': "%3A",
  ';': "%3B",
  '=': "%3D",
  '?': "%3F",
  '@': "%40",
}

def encode_s3_url(url):
    chars = "+!\"#$&\'()*,:;=?@"
    for char in chars:
        url = url.replace(char, encodings[char])
    url = url.replace("https%3A", "https:")
    return url

## Output
Create the dictionary containing all the necessary data and outputs it as a JSON file.

In [69]:
# Sort the calls by their date in descending order
calls["Items"].sort(key=sort_by_date, reverse=True)

# Get calls and messages from the database
calls, types_of_feedback = get_calls(calls)
messages, types_of_feedback = get_messages(types_of_feedback)

# Combine both calls and messages into one list
feedback = sorted(calls['Items'] + messages, key=lambda d: dt.strptime(d['callDate'], "%m/%d/%Y"))

# Statistics on both calls and messsages
total_feedback = calls["Count"] + len(messages)
callers = calls['Items']
dates = get_dates(feedback)
feedback_per_day = get_feedback_per_day(feedback, dates) 
types_of_feedback_per_day = get_types_of_feedback_per_day(feedback, dates)
service_feedback =  types_of_feedback_per_day['serviceFeedback'] 
doctor_feedback =  types_of_feedback_per_day['doctorFeedback'] 
med_feedback =  types_of_feedback_per_day['medFeedback'] 
other_feedback =  types_of_feedback_per_day['otherFeedback'] 
unknown_feedback =  types_of_feedback_per_day['unknownFeedback'] 

ivr = {
    "totalFeedback": total_feedback,
    "typesOfFeedback": types_of_feedback,
    "callers": callers,
    "messagers": messages,
    "dates": dates,
    "feedbackPerDay": feedback_per_day,
    "serviceFeedback": service_feedback, 
    "doctorFeedback": doctor_feedback, 
    "medFeedback": med_feedback, 
    "otherFeedback": other_feedback, 
    "unknownFeedback": unknown_feedback, 
    
}

with open("ivr_data.json", "w") as outfile:
    json.dump(ivr, outfile, indent=2)

#### Copy file back to s3 bucket
Let's copy the newly generated file back to the S3 bucket to refresh the view

In [70]:
# Uploads new json file to dashboard bucket
s3 = boto3.resource('s3')
BUCKET = "worldbank-fcv-dashboard"

s3.Bucket(BUCKET).upload_file("ivr_data.json", "data.json")